# Loading and preparing the dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="./vm_info.csv")
dataset

In [ ]:
columns = dataset.column_names
columns_to_keep = ['Platform', 'Distributor', 'Description', 'Min_CPU', 'Min_RAM_GB', 'Min_Storage_GB', 'Installed_Software', 'Image_Version']
columns_to_remove = set(columns_to_keep).symmetric_difference(columns['train'])
issues_dataset = dataset.remove_columns(columns_to_remove)
issues_dataset

In [ ]:
issues_dataset.set_format(type="pandas")
df = issues_dataset['train'][:]

In [ ]:
df.head()

In [ ]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(df)
comments_dataset

In [ ]:
def concatenate_text(examples):
    return {
        "text": examples["Platform"] + " " + 
               examples["Distributor"] + " " + 
               examples["Description"] + " " + 
               "Min CPU: " + str(examples["Min_CPU"]) + " cores " + 
               "Min RAM: " + str(examples["Min_RAM_GB"]) + " GB " + 
               "Min Storage: " + str(examples["Min_Storage_GB"]) + " GB " +
               "Installed Software: " + examples["Installed_Software"] + " " + 
               "Image Version: " + examples["Image_Version"]
    }

In [ ]:
comments_dataset = comments_dataset.map(concatenate_text)
comments_dataset

In [ ]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [ ]:
import torch

device = torch.device("cpu")
model.to(device)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
import pandas as pd

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"])[0]}
)

In [ ]:
embeddings_dataset.add_faiss_index(column="embeddings")

In [ ]:
question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).detach().numpy()
question_embedding.shape

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.text}")
    print(f"SCORE: {row.scores}")
    print("=" * 50)
    print()